In [ ]:
#Implement flask
import json
from flask import Flask, request
import sqlite3
import pandas as pd
CovidVaccineApp = Flask(__name__)
# CovidVaccineApp.debug = True
@CovidVaccineApp.route('/')
def status():
    return "Yaaas! the server's running"
@CovidVaccineApp.route('/search', methods=['GET'])
def search_query():
    state = request.args.get("state")
    try:
        conn = sqlite3.connect('covid_vaccine.db')
        query = "select sentiment, count(*) as count from covid_vaccine_result where sentiment is not null group by state, sentiment having state ='" +state+ "';"
        result = pd.read_sql(query, conn)
#         print(result)
        output = dict(zip(result['sentiment'], result['count']))
#         print(output)
    except:
        return "Failed!"
    return json.dumps(output, indent = 4) 

@CovidVaccineApp.route('/summarize', methods=['GET'])
def summearize_text():
    state = request.args.get("state")
    city = request.args.get("city")
    # Call Ronak's module to get the reviews (Pass city, state, neighborbood)
    # Call your review sentiment and summarization
    pass
if __name__ == '__main__':
    CovidVaccineApp.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Dec/2020 21:29:37] "GET /search?state=FL HTTP/1.1" 200 -


  sentiment  count
0  NEGATIVE     61
1  POSITIVE     37
{'NEGATIVE': 61, 'POSITIVE': 37}


127.0.0.1 - - [19/Dec/2020 21:29:54] "GET /search?state=MA HTTP/1.1" 200 -


  sentiment  count
0  NEGATIVE     66
1  POSITIVE     59
{'NEGATIVE': 66, 'POSITIVE': 59}


...

  sentiment  count
0  NEGATIVE     42
1  POSITIVE     57
{'NEGATIVE': 42, 'POSITIVE': 57}


127.0.0.1 - - [19/Dec/2020 21:30:43] "GET /search?state=FL HTTP/1.1" 200 -


  sentiment  count
0  NEGATIVE     61
1  POSITIVE     37
{'NEGATIVE': 61, 'POSITIVE': 37}


127.0.0.1 - - [19/Dec/2020 21:31:42] "GET /search?state=TN HTTP/1.1" 200 -


  sentiment  count
0  NEGATIVE     17
1  POSITIVE     52
{'NEGATIVE': 17, 'POSITIVE': 52}
